The goal of this notebook is to query the rich dataset of nycflights13, and to recreate tables and calculations from my stats306 repository problemset 2, found at https://github.com/kaspersj/Stats-306/blob/master/problemset2.ipynb

In [3]:
import pandas as pd
import numpy as np
import nycflights13 as nyc
# import pyspark
import duckdb

In [4]:
flights = nyc.flights
airlines = nyc.airlines

In [5]:
flights.size

6398744

In [6]:
flights.columns

Index(['year', 'month', 'day', 'dep_time', 'sched_dep_time', 'dep_delay',
       'arr_time', 'sched_arr_time', 'arr_delay', 'carrier', 'flight',
       'tailnum', 'origin', 'dest', 'air_time', 'distance', 'hour', 'minute',
       'time_hour'],
      dtype='object')

In [7]:
airlines.size

32

In [8]:
airlines.columns

Index(['carrier', 'name'], dtype='object')

## Problem 1:
Extract all the flights that took place in months that begin with the letter "A"

In [9]:
# flights that took place in months starting with the letter "A" occurred in either April (month 4) or August (month 8)
q1results = duckdb.query("select * from flights where month in (4, 8)") 
q1results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     4 │     1 │    454.0 │            500 │ … │     84.0 │      529 │     5 │      0 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    509.0 │            515 │ … │    194.0 │     1400 │     5 │     15 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    526.0 │            530 │ … │    206.0 │     1416 │     5 │     30 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    534.0 │            540 │ … │    152.0 │     1089 │     5 │     40 │ 2013-04-01T09:00:00Z │
│  2013 │     4 │     1 │    542.0 │    

# Problem 2

Find the number of United Airlines flights that departed exactly on time and arrived exactly on time.

In [10]:
q2result = duckdb.query("select count(*) as problemTwoAnswer from flights where (carrier = 'UA' and (dep_delay == 0 and arr_delay == 0))")
q2result

┌──────────────────┐
│ problemTwoAnswer │
│      int64       │
├──────────────────┤
│               58 │
└──────────────────┘

# Problem 3

Find all flights destined for LAX or SFO which departed more than an hour late, but did not arrive late.

In [11]:
q3results = duckdb.query("select * from flights where ((dest = 'LAX' OR dest = 'SFO') AND dep_delay >= 60 AND (arr_delay < 0))")
q3results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     4 │     5 │    932.0 │            831 │ … │    301.0 │     2454 │     8 │     31 │ 2013-04-05T12:00:00Z │
├───────┴───────┴───────┴──────────┴────────────────┴───┴──────────┴──────────┴───────┴────────┴──────────────────────┤
│ 1 rows                                                                                        19 columns (10 shown) │
└─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┘

# Problem 4

Find the number of flights in May that are missing their departure time or their arrival time, but not both. 

In [12]:
q4results = duckdb.query("SELECT count(*) as probFourCount from flights WHERE month = 5 AND ((arr_time IS NOT NULL) <> (dep_time is NOT NULL))") # <> denotes xor
q4results

┌───────────────┐
│ probFourCount │
│     int64     │
├───────────────┤
│            38 │
└───────────────┘

# Problem 5

There are four flights which have a tail number that does not begin with the letter "N". Find those 

In [13]:
q5results = duckdb.query("SELECT * FROM flights where tailnum not like 'N%'")
q5results

┌───────┬───────┬───────┬──────────┬────────────────┬───┬──────────┬──────────┬───────┬────────┬──────────────────────┐
│ year  │ month │  day  │ dep_time │ sched_dep_time │ … │ air_time │ distance │ hour  │ minute │      time_hour       │
│ int64 │ int64 │ int64 │  double  │     int64      │   │  double  │  int64   │ int64 │ int64  │       varchar        │
├───────┼───────┼───────┼──────────┼────────────────┼───┼──────────┼──────────┼───────┼────────┼──────────────────────┤
│  2013 │     2 │    11 │   1508.0 │           1400 │ … │    131.0 │      762 │    14 │      0 │ 2013-02-11T19:00:00Z │
│  2013 │     3 │    23 │   1340.0 │           1300 │ … │    153.0 │      950 │    13 │      0 │ 2013-03-23T17:00:00Z │
│  2013 │     3 │    24 │    859.0 │            835 │ … │    143.0 │      944 │     8 │     35 │ 2013-03-24T12:00:00Z │
│  2013 │     7 │     5 │   1253.0 │           1259 │ … │    112.0 │      762 │    12 │     59 │ 2013-07-05T16:00:00Z │
├───────┴───────┴───────┴──────────┴────

# Problem 6

Find the unique tail numbers(s) of the flights that were in the air for the second shortest and third longest amounts of time. Store them in variables shortest2_tail_num and longest3_tail_num, respectively.

--> Running into the issue: there is no guarantee that an R function can be translated into sql code
- To solve this issue, instead of using R's mutate and min_rank, I found the second shortest and third longest air_time values and plugged those into a where clause

In [14]:
q6results_secondShortest = duckdb.query("SELECT distinct(tailnum) from flights WHERE air_time = (SELECT min(air_time) as secondShortest from flights where air_time > (select min(air_time) from flights))")
    #"SELECT * FROM (SELECT DISTINCT(tailnum), row_number() OVER (ORDER BY air_time ASC) AS RN from flights) WHERE RN = 2")
    #"SELECT distinct(tailnum), nth_value(tailnum, 2) OVER (ORDER BY air_time ASC) from flights")
    #"select distinct(tailnum), (nth_value(tailnum, 2) OVER (ORDER BY air_time ASC), nth_value(tailnum, 2) OVER (ORDER BY tailnum ROWS 2 PRECEDING) FROM flights")
q6results_secondShortest

┌─────────┐
│ tailnum │
│ varchar │
├─────────┤
│ N947UW  │
│ N12195  │
│ N14920  │
│ N13955  │
│ N8501F  │
│ N16987  │
│ N11191  │
│ N12160  │
│ N11109  │
│ N12921  │
│ N29906  │
│ N13913  │
│ N27200  │
│ N18101  │
├─────────┤
│ 14 rows │
└─────────┘

In [15]:
q6results_thirdLongest = duckdb.query("SELECT distinct(tailnum) from flights WHERE air_time = (SELECT air_time from flights ORDER BY air_time DESC LIMIT 1 OFFSET 2)")
q6results_thirdLongest

┌─────────┐
│ tailnum │
│ varchar │
├─────────┤
│ N380HA  │
│ N388HA  │
└─────────┘

# Problem 7

Sort the rows of flights such that the months are arranged in the following order: spring, summer, fall, winter. (Here we define winter to be January-March, spring is April-June, etc.) Within each season, the months should be sorted in ascending order. After sorting, drop all columns except for month, day and tail number. Store the sorted and subsetted table in a variable called table7.

In [16]:
q7result = duckdb.query("SELECT month, day, tailnum from flights ORDER BY case month \
                            when '4' then 1 \
                            when '5' then 2 \
                            when '6' then 3 \
                            when '7' then 4 \
                            when '8' then 5 \
                            when '9' then 6 \
                            when '10' then 7 \
                            when '11' then 8 \
                            when '12' then 9 \
                            when '1' then 10 \
                            when '2' then 11 \
                            when '3' then 12 \
                        end, \
                        month")
q7result

┌───────┬───────┬─────────┐
│ month │  day  │ tailnum │
│ int64 │ int64 │ varchar │
├───────┼───────┼─────────┤
│     4 │     1 │ N566UW  │
│     4 │     1 │ N76288  │
│     4 │     1 │ N76517  │
│     4 │     1 │ N5DSAA  │
│     4 │     1 │ N784JB  │
│     4 │     1 │ N590JB  │
│     4 │     1 │ N737US  │
│     4 │     1 │ N750UW  │
│     4 │     1 │ N510MQ  │
│     4 │     1 │ N723EV  │
│     · │     · │   ·     │
│     · │     · │   ·     │
│     · │     · │   ·     │
│     4 │    11 │ N826AS  │
│     4 │    11 │ N11137  │
│     4 │    11 │ N604DL  │
│     4 │    11 │ N17126  │
│     4 │    11 │ N5CHAA  │
│     4 │    11 │ N584AA  │
│     4 │    11 │ N249JB  │
│     4 │    11 │ N284WN  │
│     4 │    11 │ N823UA  │
│     4 │    11 │ N17105  │
├───────┴───────┴─────────┤
│ ? rows        3 columns │
└─────────────────────────┘

# Problem 8

Drop the even-numbered columns of flights. Store the resulting data table in a variable called table8.

In [17]:
q8result = duckdb.query("SELECT year, day, sched_dep_time, arr_time, arr_delay, flight, origin, air_time, hour, time_hour from flights") 
# no function to drop even numbered cols in SQL, so I hard-coded this one
q8result

┌───────┬───────┬────────────────┬──────────┬───────────┬────────┬─────────┬──────────┬───────┬──────────────────────┐
│ year  │  day  │ sched_dep_time │ arr_time │ arr_delay │ flight │ origin  │ air_time │ hour  │      time_hour       │
│ int64 │ int64 │     int64      │  double  │  double   │ int64  │ varchar │  double  │ int64 │       varchar        │
├───────┼───────┼────────────────┼──────────┼───────────┼────────┼─────────┼──────────┼───────┼──────────────────────┤
│  2013 │     1 │            515 │    830.0 │      11.0 │   1545 │ EWR     │    227.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            529 │    850.0 │      20.0 │   1714 │ LGA     │    227.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            540 │    923.0 │      33.0 │   1141 │ JFK     │    160.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            545 │   1004.0 │     -18.0 │    725 │ JFK     │    183.0 │     5 │ 2013-01-01T10:00:00Z │
│  2013 │     1 │            600 │    812.0 │   

# Problem 9

Define a flight to be "spooky" if it was in the air at 1:13pm on either Halloween or Friday, September 13th. There were 110 spooky flights on Friday the 13th. How many spooky flights are there in total? Store your answer in a variable called nspooky.

In [18]:
q9result = duckdb.query("SELECT count(*) as nspooky from flights where ((day = 13 OR day = 31) AND (month = 9 or month = 10) AND (1313 < arr_time AND 1313 > dep_time))")
q9result

┌─────────┐
│ nspooky │
│  int64  │
├─────────┤
│     318 │
└─────────┘

# Problem 10

Select all the columns in flights which end with "delay". In the resulting table, define a new column max_delay which equals the maximum of the departure and arrival delays for each flight. For example, if a flight had a departure delay of -1 and an arrival delay of 10, the max_delay would equal 10. Sort your table in descending order of max_delay and store it in a variable called table10.

In [19]:
q10result = duckdb.query("SELECT dep_delay, arr_delay, GREATEST(dep_delay, arr_delay) as max_delay from flights ORDER BY max_delay DESC ") # unfortunately the LIKE operator works within columns, not on column names
q10result

┌───────────┬───────────┬───────────┐
│ dep_delay │ arr_delay │ max_delay │
│  double   │  double   │  double   │
├───────────┼───────────┼───────────┤
│    1301.0 │    1272.0 │    1301.0 │
│    1137.0 │    1127.0 │    1137.0 │
│    1126.0 │    1109.0 │    1126.0 │
│    1014.0 │    1007.0 │    1014.0 │
│    1005.0 │     989.0 │    1005.0 │
│     960.0 │     931.0 │     960.0 │
│     911.0 │     915.0 │     915.0 │
│     899.0 │     850.0 │     899.0 │
│     898.0 │     895.0 │     898.0 │
│     896.0 │     878.0 │     896.0 │
│        ·  │       ·   │       ·   │
│        ·  │       ·   │       ·   │
│        ·  │       ·   │       ·   │
│      38.0 │     129.0 │     129.0 │
│     105.0 │     129.0 │     129.0 │
│     129.0 │     110.0 │     129.0 │
│     129.0 │     106.0 │     129.0 │
│     114.0 │     129.0 │     129.0 │
│     105.0 │     129.0 │     129.0 │
│     129.0 │      86.0 │     129.0 │
│     129.0 │     117.0 │     129.0 │
│     129.0 │     101.0 │     129.0 │
│     117.0 

This is the end of this notebook.